In [17]:
#Author: Aline Andrade (@souzalii)

# This script reads an Excel file containing researcher information, cleans and standardizes the data by removing unwanted keywords,
# and consolidates organization names. It also merges rows based on 'Name' and 'Organisation' while combining relevant fields.

In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
#read file
df = pd.read_excel('path/to/your/dummy_data.xlsx')

In [3]:
# Remove leading and trailing spaces in the columns
# Note: change the columns name as per your file
df['Name'] = df['Name'].str.strip()
df['Organisation'] = df['Organisation'].str.strip()
df['Listed as'] = df['Listed as'].str.strip()

In [4]:
# Function to remove parentheses and the content within them
def remove_parentheses(text):
    if isinstance(text, str):  
        return re.sub(r'\s*\(.*?\)\s*', '', text)
    return text  

# Convert all values to string to avoid errors
df['Name'] = df['Name'].astype(str).apply(remove_parentheses)

In [5]:
# Keywords to be removed
# Note: insert the words for which you want to remove the entire row if it contains any of these keywords
keywords = ['TBA', 'TBC', 'TBD', 'staff']

# Function to check if the text contains any of the keywords
def contains_keywords(text):
    if isinstance(text, str):
        pattern = '|'.join(keywords)
        return not re.search(pattern, text, re.IGNORECASE)
    return False

# Filter the DataFrame to remove rows containing the keywords
df = df[df['Name'].apply(contains_keywords)]

In [6]:
# Exact values to be removed
# Note: this will remove the rows that exactly match the words
values_to_remove = [
    'Postdoctoral fellow',
    'Up to 5 sub-project leads',
]

# Filter the DataFrame to remove rows with the exact values
df = df[~df['Name'].isin(values_to_remove)]

In [7]:
# Dictionary of substitutions to standardize organization names
organization_mapping = {
    "UN": "University of Newcastle",
    "University of Queensland": "The University of Queensland",
    "UNE": "University of New England",
}

# Function to standardize organization names
def standardize_organization(name):
    return organization_mapping.get(name, name)

# Apply the standardization to the organization columns
df['Organisation'] = df['Organisation'].apply(standardize_organization)

In [10]:
# Define a function to merge rows 
def merge_rows(group):
    listed_as = ' '.join(group['Listed as'].dropna().unique())
    orcid = ' '.join(group['ORCID'].dropna().unique())
    url = ' '.join(group['URL'].dropna().unique())
    return pd.Series([listed_as, orcid, url])

# Apply the merging function
merged_df = df.groupby(['Name', 'Organisation'], as_index=False).apply(merge_rows, include_groups=False)

In [12]:
# Rename the columns back
merged_df.columns = ['Name', 'Organisation', 'Listed as', 'ORCID', 'URL']

In [14]:
# Function to handle non-string values and remove duplicates
def combine_series(series):
    return '; '.join(sorted(set(str(x) for x in series if pd.notna(x))))

# Group by 'Name' and combine 'Organisation' and 'Listed as' with ';' as a separator
merged_df = df.groupby('Name').agg({
    'Organisation': combine_series,
    'Listed as': combine_series,
    'ORCID': 'first',  
    'URL': 'first'     
}).reset_index()

In [16]:
# Save the result to a new Excel file
merged_df.to_excel('resultado_mesclado.xlsx', index=False)